ALS necesita que los identificadores sean valores enteros. Hay que hacer un diccionario para poder utilizar el algoritmo ALS. Usar stringindex + pipeline. 

Dado el dataset ./datos/lastfm_mini.parquet, se pide:

Imprimir por pantalla los 20 artistas que más aparecen. Agrupar por identificar de artista y hacer un sum. 

Crear un sistema de recomendación basado en el algortimo ALS visto en el notebook 04.ALS_Spark.ipynb

Calcular el usuario más a fín a el grupo ramones según nuestro modelo

Para el usuario del punto anterior conseguir sus 20 artístas más afin según nuestro modelo

Se valorará conseguir un modelo adecuado usando las técnicas vistas en clase

In [3]:
import os
import sys

# Configuration properties
APP_NAME = 'interactive-script'
MASTER = 'local[*]'
SPARK_HOME = 'C:\spark-2.3.1-bin-hadoop2.7'
#SPARK_HOME = 'C:\spark-2.2.0-bin-hadoop2.7'
HADOOP_HOME = 'C:\SparkClase\hadoop'
PYSPARK_LIB = 'pyspark.zip'
PY4J_LIB = 'py4j-0.10.7-src.zip'
#PY4J_LIB = 'py4j-0.10.4-src.zip'

# Spark initialization
os.environ['SPARK_HOME'] = SPARK_HOME
os.environ['HADOOP_HOME'] = HADOOP_HOME
sys.path.insert(0, os.path.join(SPARK_HOME, 'python'))
sys.path.insert(0, os.path.join(SPARK_HOME, 'python', 'lib'))
sys.path.insert(0, os.path.join(SPARK_HOME, 'python', 'lib', PYSPARK_LIB))
sys.path.insert(0, os.path.join(SPARK_HOME, 'python', 'lib', PY4J_LIB))

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName(APP_NAME)
conf = conf.setMaster(MASTER)
sparkSession = SparkSession.builder.config(conf = conf).getOrCreate()
sparkContext = sparkSession.sparkContext

In [4]:
import pandas as pd
import numpy as np

from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [5]:
plays = sparkSession.read.parquet('datos/lastfm_mini.parquet')

In [6]:
plays.show()

+--------------------+--------------------+-----------+-----+
|             user_id|           artist_id|artist_name|plays|
+--------------------+--------------------+-----------+-----+
|8358173502fcfd30b...|382f1005-e9ab-468...|       2pac| 54.0|
|f434362a46b823536...|382f1005-e9ab-468...|       2pac|545.0|
|135750e1a2107ae20...|382f1005-e9ab-468...|       2pac| 54.0|
|41a072560f38290a2...|382f1005-e9ab-468...|       2pac|201.0|
|57da4f4db6fc10eb7...|382f1005-e9ab-468...|       2pac| 86.0|
|3f51326aec6893f15...|382f1005-e9ab-468...|       2pac| 95.0|
|ac4f934779afac87b...|382f1005-e9ab-468...|       2pac|163.0|
|858ebba231d365236...|382f1005-e9ab-468...|       2pac| 99.0|
|24efaa9b0170cda92...|382f1005-e9ab-468...|       2pac| 71.0|
|dd5b2d9f5d10d428d...|382f1005-e9ab-468...|       2pac| 18.0|
|7550b588c297f6792...|382f1005-e9ab-468...|       2pac| 47.0|
|662eee6265ed61f4a...|382f1005-e9ab-468...|       2pac|257.0|
|6e7acb2ec69628dc6...|382f1005-e9ab-468...|       2pac|388.0|
|45d4068

In [7]:
plays.groupBy(["artist_id","artist_name"]).sum().orderBy("sum(plays)", ascending = False).show(20)

+--------------------+--------------------+----------+
|           artist_id|         artist_name|sum(plays)|
+--------------------+--------------------+----------+
|b10bbbfc-cf9e-42e...|         the beatles|  878602.0|
|a74b1b7f-71a5-401...|           radiohead|  840236.0|
|83d91898-7763-47d...|          pink floyd|  492270.0|
|cc197bad-dc9c-440...|            coldplay|  489212.0|
|65f4f0c5-ef9e-490...|           metallica|  472438.0|
|9c9f1380-2516-4fc...|                muse|  449794.0|
|8bfac288-ccc5-448...|red hot chili pep...|  413867.0|
|b7ffd2af-418f-4be...|     nine inch nails|  402385.0|
|cc0b7089-c08d-4c1...|    system of a down|  394673.0|
|00a9f935-ba93-4fc...|           nightwish|  384564.0|
|f59c5520-5f46-4d2...|         linkin park|  361024.0|
|31745282-b1ea-4d6...|           in flames|  351163.0|
|0039c7ae-e1a7-4a7...| death cab for cutie|  327846.0|
|678d88b2-87b0-403...|        led zeppelin|  304345.0|
|847e8284-8582-4b0...|             placebo|  303084.0|
|72c536dc-

In [8]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [9]:
indexer_user = StringIndexer(inputCol="user_id", outputCol="user_id_index")
indexer_artist = StringIndexer(inputCol="artist_id", outputCol="artist_id_index")

In [10]:
Model = ALS(

    regParam=0.01, #mallado.iloc[0]['regParam'],
    maxIter=5, #mallado.iloc[0]['maxIter'],
    rank=10, #mallado.iloc[0]['rank'],    
    userCol="user_id_index", 
    itemCol="artist_id_index", 
    ratingCol="plays",
    coldStartStrategy="drop",
    implicitPrefs=False,
    numUserBlocks=10,
    numItemBlocks=10

)

In [11]:
pipeline = Pipeline(stages=[indexer_user, indexer_artist, Model])

In [12]:
model_fit = pipeline.fit(plays)

In [13]:
predictions = model_fit.transform(plays)

In [14]:
predictions.show(20)

+--------------------+--------------------+-----------+-----+-------------+---------------+----------+
|             user_id|           artist_id|artist_name|plays|user_id_index|artist_id_index|prediction|
+--------------------+--------------------+-----------+-----+-------------+---------------+----------+
|b7208c428e0b4cca2...|148ddea2-6839-435...|tenacious d|225.0|        737.0|          148.0| 554.21094|
|c49aa95507e4888af...|148ddea2-6839-435...|tenacious d|611.0|       1084.0|          148.0| 552.99445|
|34ee8348c1c9ee29a...|148ddea2-6839-435...|tenacious d|361.0|       1522.0|          148.0|  738.6395|
|ece8e94f1e295064f...|148ddea2-6839-435...|tenacious d| 87.0|       1618.0|          148.0| 94.969246|
|90d121e0b0315f2b0...|148ddea2-6839-435...|tenacious d|107.0|       2711.0|          148.0| 141.47751|
|721073ef1bcf40801...|148ddea2-6839-435...|tenacious d|104.0|       4489.0|          148.0| 85.059616|
|28da06bbebb9c564b...|148ddea2-6839-435...|tenacious d| 77.0|        451.

In [15]:
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="plays",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 186.54708280424938


In [16]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [17]:
(training, test) = plays.randomSplit([0.8, 0.2], seed=12)

In [18]:
als = ALS(

    userCol="user_id_index", 
    itemCol="artist_id_index", 
    ratingCol="plays",
    coldStartStrategy="drop",
    implicitPrefs=False,
    numUserBlocks=10,
    numItemBlocks=10

)

In [19]:
paramGrid = (

    ParamGridBuilder()
    .addGrid(als.rank, [10, 20])
    .addGrid(als.regParam, [0.01, 0.1])
    .addGrid(als.maxIter, [5, 10])
    .build()

)

In [20]:
pipeline = Pipeline(stages=[indexer_user, indexer_artist, als])

In [21]:
import time

In [22]:
crossval = CrossValidator(

    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=2

) 

In [23]:
t0 = time.time()
cvModel = crossval.fit(training)
print(time.time()-t0)

Py4JJavaError: An error occurred while calling o229.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 138.0 failed 1 times, most recent failure: Lost task 0.0 in stage 138.0 (TID 1597, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:148)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleMapT

In [ ]:
sparkSession.stop()